In [2]:
# ============================================================================
# Mistral 7B v0.3 Activation Extraction (Kaggle Version)
# ============================================================================

# -------- INSTALL --------
!pip install -q -U transformers accelerate bitsandbytes huggingface_hub

# -------- IMPORTS --------
import os
import glob
import numpy as np
import pandas as pd
import torch
import shutil
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# -------- CONFIG --------
# CHANGED: Updated to Mistral v0.3
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.3"

# Auto-detect input CSV path
input_files = glob.glob("/kaggle/input/**/queries_400.csv", recursive=True)
if input_files:
    QUERIES_PATH = input_files[0]
    print(f"✅ Found input file: {QUERIES_PATH}")
else:
    raise FileNotFoundError("❌ Could not find 'queries_400.csv' in /kaggle/input.")

OUTPUT_DIR = "/kaggle/working/ACTIVATIONS"
# Mistral has 32 layers. Layers 3,4,5,6 are valid low-level features.
LAYER_INDICES = [3, 4, 5, 6] 
MAX_LEN = 1024
FWD_BATCH = 4
SAVE_BATCH = 40

os.makedirs(OUTPUT_DIR, exist_ok=True)

# -------- KAGGLE SECRETS AUTH --------
try:
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("HF_TOKEN")
    login(token=hf_token)
    print("✅ Logged into HF via Kaggle Secrets")
except Exception as e:
    print(f"❌ Auth failed. Check 'HF_TOKEN' in Add-ons -> Secrets. Error: {e}")

# -------- LOAD MODEL (4-BIT QUANTIZED) --------
print(f"🚀 Loading {MODEL_ID} with 4-bit quantization...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=hf_token)

# Mistral often lacks a default pad token, setting it to EOS is standard practice
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    token=hf_token,
    device_map="auto",
    output_hidden_states=True,
    quantization_config=quant_config,
    trust_remote_code=True
)
model.eval()
print(f"✅ Model loaded: {next(model.parameters()).device}")

# -------- LOAD QUERIES --------
df = pd.read_csv(QUERIES_PATH)
if 'query_id' not in df.columns:
    df['query_id'] = range(len(df))
df = df.sort_values('query_id').reset_index(drop=True)
queries = df['query_text'].astype(str).tolist()
print(f"✅ Loaded {len(queries)} queries")

# -------- VALIDATE LAYERS --------
num_layers = model.config.num_hidden_layers
valid_layers = [li for li in LAYER_INDICES if li + 1 < num_layers]
print(f"✅ Valid layers: {valid_layers}")

# -------- EXTRACTION --------
@torch.inference_mode()
def extract_hidden_batch(text_list):
    enc = tokenizer(text_list, return_tensors="pt", padding=True, 
                    truncation=True, max_length=MAX_LEN).to(model.device)
    out = model(**enc, output_hidden_states=True, return_dict=True)
    hidden = out.hidden_states
    
    pooled_layers = []
    for li in valid_layers:
        # Mistral structure is similar to Llama: hidden[0] is embeddings
        layer_tensor = hidden[li + 1] 
        pooled = layer_tensor.mean(dim=1)
        pooled_layers.append(pooled.cpu().numpy())
    
    return np.stack(pooled_layers, axis=1).astype(np.float32)

# -------- MAIN LOOP --------
print(f"\n🎯 Extracting activations for Mistral...")
meta_rows = []
batch_idx = 0
safe_model_name = MODEL_ID.replace("/", "_")

for start_idx in range(0, len(queries), SAVE_BATCH):
    end_idx = min(len(queries), start_idx + SAVE_BATCH)
    batch_queries = queries[start_idx:end_idx]
    
    sub_outputs = []
    for i in range(0, len(batch_queries), FWD_BATCH):
        sub_texts = batch_queries[i:i+FWD_BATCH]
        try:
            acts = extract_hidden_batch(sub_texts)
            sub_outputs.append(acts)
        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                print(f"  OOM, fallback to single...")
                for single in sub_texts:
                    sub_outputs.append(extract_hidden_batch([single]))
            else:
                raise
    
    batch_acts = np.vstack(sub_outputs)
    
    # Save
    npz_name = f"{safe_model_name}_acts_{batch_idx:03d}_{start_idx}_{end_idx-1}.npz"
    npz_path = os.path.join(OUTPUT_DIR, npz_name)
    np.savez_compressed(
        npz_path,
        activations=batch_acts,
        query_ids=df.iloc[start_idx:end_idx]['query_id'].to_numpy(),
        categories=df.iloc[start_idx:end_idx].get('category', ['unknown']*(end_idx-start_idx)).to_numpy()
    )
    print(f"  ✅ Batch {batch_idx}: {npz_name} ({os.path.getsize(npz_path)/1e6:.1f}MB)")
    
    # Metadata
    for local_idx, qid in enumerate(df.iloc[start_idx:end_idx]['query_id'].tolist()):
        meta_rows.append({
            "query_id": int(qid),
            "npz_file": npz_name,
            "local_index": int(local_idx),
            "category": df.iloc[start_idx + local_idx].get('category', 'unknown')
        })
    
    batch_idx += 1

# -------- SAVE INDEX & ZIP --------
meta_df = pd.DataFrame(meta_rows)
index_csv = os.path.join(OUTPUT_DIR, f"{safe_model_name}_activation_index.csv")
meta_df.to_csv(index_csv, index=False)

print(f"\n✅ COMPLETE: Output saved to {OUTPUT_DIR}")

print("📦 Zipping output for download...")
shutil.make_archive("/kaggle/working/mistral_activations_output", 'zip', OUTPUT_DIR)
print("✅ Ready! Download 'mistral_activations_output.zip' from the Output tab.")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


✅ Found input file: /kaggle/input/hallucination-detection-dataset-400/queries_400.csv
✅ Logged into HF via Kaggle Secrets
🚀 Loading mistralai/Mistral-7B-Instruct-v0.3 with 4-bit quantization...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

✅ Model loaded: cuda:0
✅ Loaded 400 queries
✅ Valid layers: [3, 4, 5, 6]

🎯 Extracting activations for Mistral...
  ✅ Batch 0: mistralai_Mistral-7B-Instruct-v0.3_acts_000_0_39.npz (1.6MB)
  ✅ Batch 1: mistralai_Mistral-7B-Instruct-v0.3_acts_001_40_79.npz (1.6MB)
  ✅ Batch 2: mistralai_Mistral-7B-Instruct-v0.3_acts_002_80_119.npz (1.6MB)
  ✅ Batch 3: mistralai_Mistral-7B-Instruct-v0.3_acts_003_120_159.npz (1.6MB)
  ✅ Batch 4: mistralai_Mistral-7B-Instruct-v0.3_acts_004_160_199.npz (1.6MB)
  ✅ Batch 5: mistralai_Mistral-7B-Instruct-v0.3_acts_005_200_239.npz (1.6MB)
  ✅ Batch 6: mistralai_Mistral-7B-Instruct-v0.3_acts_006_240_279.npz (1.6MB)
  ✅ Batch 7: mistralai_Mistral-7B-Instruct-v0.3_acts_007_280_319.npz (1.6MB)
  ✅ Batch 8: mistralai_Mistral-7B-Instruct-v0.3_acts_008_320_359.npz (1.6MB)
  ✅ Batch 9: mistralai_Mistral-7B-Instruct-v0.3_acts_009_360_399.npz (1.6MB)

✅ COMPLETE: Output saved to /kaggle/working/ACTIVATIONS
📦 Zipping output for download...
✅ Ready! Download 'mistral_activ